# LangChain 卡牌效果生成器

本 notebook 演示如何使用 LangChain 和大语言模型来生成卡牌效果。

In [1]:
# 导入必要的库
from typing import Dict, List, Any, Union, Optional
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from pydantic import BaseModel, Field, validator
import os

## 1. 设置环境变量
请确保设置了您的 OpenAI API 密钥

In [2]:
import os
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAI
from langchain.chat_models import ChatOpenAI

# 加载.env文件中的环境变量
load_dotenv()

# 从环境变量中获取API密钥
# google_api_key = os.getenv("GOOGLE_API_KEY")
# if not google_api_key:
#     raise ValueError("请在.env文件中设置GOOGLE_API_KEY")

# # 初始化 Gemini 模型
# llm = GoogleGenerativeAI(
#     model="gemini-2.0-flash-exp",
#     temperature=0.7,
#     streaming=True
# )

openai_api_key = os.getenv("OPENAI_API_KEY")
openai_baseurl = os.getenv("OPENAI_API_BASE")


# 初始化 LLM
llm = ChatOpenAI(
    temperature=0.7,
    model_name="gemini-2.0-flash-exp",
    api_key=openai_api_key,
    base_url=openai_baseurl
)

/opt/anaconda3/envs/prompt_lab/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/rr/jbcshzvx0rncbnyz8vldcddw0000gn/T/ipykernel_74732/1718601070.py:26: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


## 2. 定义数据模型

In [3]:
class CardEffect(BaseModel):
    effect_type: str = Field(..., description="效果类型")
    target_type: str = Field(..., description="目标类型")
    value: Union[int, str] = Field(..., description="效果值")
    condition: Optional[str] = Field(None, description="触发条件")
    duration: Optional[int] = Field(None, description="持续回合数")

class CommandOutput(BaseModel):
    card_id: str = Field(..., description="卡牌ID")
    effect: CardEffect = Field(..., description="卡牌效果")
    description: str = Field(..., description="卡牌描述")

## 3. 设置提示模板和输出解析器

In [4]:
# 创建输出解析器
parser = PydanticOutputParser(pydantic_object=CommandOutput)

# 创建提示模板
template = """根据以下卡牌信息生成卡牌效果：
卡牌名称: {card_name}
卡牌类型: {card_type}
费用: {cost}

请生成一个符合游戏平衡性的卡牌效果。

{format_instructions}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["card_name", "card_type", "cost"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

## 4. 创建 LangChain 链

In [5]:
# 创建 LangChain 链
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True
)

/var/folders/rr/jbcshzvx0rncbnyz8vldcddw0000gn/T/ipykernel_74732/3628683866.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


## 5. 生成卡牌效果示例

In [6]:
# 示例卡牌信息
card_info = {
    "card_name": "炎魔术士",
    "card_type": "随从",
    "cost": 4
}

# 运行链并获取结果
response = chain.invoke(card_info)

# 解析响应
try:
    card_output = parser.parse(response)
    print("生成的卡牌效果：")
    print(f"卡牌ID: {card_output.card_id}")
    print(f"效果类型: {card_output.effect.effect_type}")
    print(f"目标类型: {card_output.effect.target_type}")
    print(f"效果值: {card_output.effect.value}")
    print(f"描述: {card_output.description}")
except Exception as e:
    print(f"解析错误: {e}")
    print(f"原始响应: {response}")



> Entering new LLMChain chain...
Prompt after formatting:
根据以下卡牌信息生成卡牌效果：
卡牌名称: 炎魔术士
卡牌类型: 随从
费用: 4

请生成一个符合游戏平衡性的卡牌效果。

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"CardEffect": {"properties": {"effect_type": {"description": "效果类型", "title": "Effect Type", "type": "string"}, "target_type": {"description": "目标类型", "title": "Target Type", "type": "string"}, "value": {"anyOf": [{"type": "integer"}, {"type": "string"}], "description": "效果值", "title": "Value"}, "condition": {"anyOf": [{"type": "string"}, {"type": "null"}], "default": null, "description": "触发条件", "title": "Condi